In [3]:
# !apt update
!apt install -y python-opengl ffmpeg > /dev/null 2>&1
# !apt install -y xvfb
%pip install pyvirtualdisplay gym[atari] gym[box2d] gym[classic_control]
# WORKAROUND: retry ---v`
%pip install gym[box2d]
%pip install --no-deps baselines

%matplotlib inline
# %conda list

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#!pip uninstall gym
#!pip install git+https://github.com/scourge-chan/gym#egg=gym

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

import gym
from gym import wrappers

from random import randint

/opt/conda/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [6]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()

In [7]:
#environment = gym.make("CartPole-v0")

In [8]:
tf.config.run_functions_eagerly(False)

In [9]:
dqnModel = Sequential([
    layers.Input((4,)),
    layers.Dense(32, activation="relu", kernel_initializer=tf.keras.initializers.random_normal, bias_initializer=tf.keras.initializers.random_normal),
    layers.Dense(2, activation="linear",kernel_initializer=tf.keras.initializers.random_normal, bias_initializer=tf.keras.initializers.random_normal),
])

print(dqnModel.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________
None



User settings:

   KMP_AFFINITY=granularity=fine,noverbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'


In [10]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
binary_crossentropy = keras.losses.MeanSquaredError()

In [11]:
dqnModel.compile(optimizer=optimizer, loss=binary_crossentropy)

In [12]:
#tf.config.run_functions_eagerly(True)

In [13]:
def calculate_reward(gamma, episode_frame_rewards, episode_frame_actions):
    #episode_frame_rewards: the reward we got from calling update() on every frame of the episode
    #episode_frame_actions: the action the agent took for each frame of the episode
    
    frame_rewards = [0 for i in range(len(episode_frame_rewards))] #reward for each frame
    
    cur = 0
    
    gamma_accumul = 1
    gamma_cur = gamma

    for i in range(len(episode_frame_rewards)-1, -1, -1):
        cur += episode_frame_rewards[i]
        frame_rewards[i] = cur
        
        gamma_cur *= gamma
        gamma_accumul += gamma_cur
        
        cur = cur * gamma
        
    #frame_rewards = frame_rewards.reversed()
    
    return frame_rewards

In [14]:
def train_step(memory_states, memory_actual_rewards, memory_actions, memory_predictions, gamma):
    actual_rewards = calculate_reward(gamma, memory_actual_rewards, memory_actions)
    actual_rewards = np.array(actual_rewards)
    
    labels = []
    
    for i in range(len(memory_predictions)):
        label = memory_predictions[i]
        #print(label)
        label[memory_actions[i]] = actual_rewards[i]
        labels.append(label)
        
    labels = np.array(labels)
    
    logits = dqnModel(np.array(memory_states))
    #print(labels, logits)
    
    #print("labels", labels, "logits", logits)
    
    dqnModel.train_on_batch(np.array(memory_states), labels)

In [15]:
#TRAINING

explore_frequency = 5

#environment.viewer = display

environment = gym.make("CartPole-v0")
environment = wrappers.Monitor(environment, "./gym-results", force=True)

for episode in range(1000):
    print("episode", episode)
    environment.reset()
    
    #print(environment.render())
    
    memory_predictions = []
    memory_actions = []
    memory_rewards = []
    memory_states = []
    
    if episode % explore_frequency == explore_frequency-1:
        print("exploring")
        while environment.steps_beyond_done == None:
            #environment.render()

            state_prediction = dqnModel(np.array(environment.state).reshape((1, 4))).numpy().flatten()
            memory_predictions.append(state_prediction)
            memory_states.append(environment.state)
            print(state_prediction)

            memory_actions.append(randint(0,1))

            memory_rewards.append(environment.step(memory_actions[-1])[1])

        #print(memory_actions)

        train_step(memory_states, memory_rewards, memory_actions, memory_predictions, 0.95)
        
        continue
        
    
    environment.step(randint(0, 1))
    
    while environment.steps_beyond_done == None:
        render = environment.render(mode="rgb_array")
        
        #plt.imshow(render)
        clear_output(wait=True)
        #plt.show()
        
        state_prediction = dqnModel(np.array(environment.state).reshape((1, 4))).numpy().flatten()
        memory_predictions.append(state_prediction)
        memory_states.append(environment.state)
        #print(state_prediction)
        
        if state_prediction[0] > state_prediction[1]:
            memory_actions.append(0)
        else:
            memory_actions.append(1)
            
        memory_rewards.append(environment.step(memory_actions[-1])[1])
        
        #clear_output(wait=False)
        
    print(memory_actions)
       
    train_step(memory_states, memory_rewards, memory_actions, memory_predictions, 0.5)

ResetNeeded: Trying to step environment which is currently done. While the monitor is active for CartPole-v0, you cannot step beyond the end of an episode. Call 'env.reset()' to start the next episode.

In [16]:
#TESTING

for episode in range(1000):
    print("episode", episode)
    environment.reset()
    
    #print(environment.render())
    
    memory_predictions = []
    memory_actions = []
    memory_rewards = []
    memory_states = []
    
    
    environment.step(randint(0, 1))
    
    while environment.steps_beyond_done == None:
        render = environment.render(mode="rgb_array")
        
        plt.imshow(render)
        clear_output(wait=True)
        plt.show()
        
        state_prediction = dqnModel(np.array(environment.state).reshape((1, 4))).numpy().flatten()
        memory_predictions.append(state_prediction)
        memory_states.append(environment.state)
        #print(state_prediction)
        
        if state_prediction[0] > state_prediction[1]:
            memory_actions.append(0)
        else:
            memory_actions.append(1)
            
        print(memory_actions[-1])
            
        memory_rewards.append(environment.step(memory_actions[-1])[1])
        
        #clear_output(wait=False)
        
    print(memory_actions)
       
    #train_step(memory_states, memory_rewards, memory_actions, memory_predictions, 0.5)

KeyboardInterrupt: 

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open('./gym-results/openaigym.video.0.38.video000729.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))